### How to get ARLIE data
In the following, you can find a beginner-friendly guide on how to retrieve ARLIE data.

[The Copernicus Aggregated River and Lake Ice Extent (ARLIE) product](https://land.copernicus.eu/en/products/water-bodies/high-resolution-aggregated-river-and-lake-ice-extent) is a spatially aggregated information on surface water conditions of rivers and lakes. ARLIE information is stored in a geodatabase, enriched every day from the River and Lake Ice extent products (RLIE S1, RLIE S2 and RLIE S1+S2) for the entire EEA38 and the United Kingdom. It provides percent coverage of snow-covered or snow-free ice on lakes and on 10 km river sections described by the EU-HYDRO river and lake network database.<br>
<br>
The ARLIE products are stored in a PostGIS persistent geodatabase. They can be retrieved by using a specific REST API. Users can query ice sheet summary information of river segments and lakes (ARLIE statistics) together with the geometry and caracteristics of the features on which these statistics were estimated. All geometry features are delivered in the ETRS89 LAEA (EPSG:3035) coordinate reference system.<br>
<br>
ARLIE is one of the products of the pan-European High-Resolution Snow & Ice service (HR-S&I), which are provided at high spatial resolution (20 m x 20 m and 60 m x 60 m) from the Sentinel-2 and Sentinel-1 constellations data from September 1, 2016 onwards.<br>
<br>
This notebook contains the following steps:<br>
1) Initial setup
2) Pick your AOI
3) Import your geometry and display it
4) Transform to WKT format
5) Request ARLIE data
6) Display ARLIE data
7) Display EU Hydro data

It is important to note that data requests must adhere to the Well-Known Text (WKT) format.<br>
<br>
This notebook is sligthly modified version of [how-to-get-started notebook](https://github.com/eea/clms-hrsi-arlie-use-case/blob/main/how-to-get-started.ipynb) developed by EEA.


#### **Step 1: Initial setup**
1. First of all, pick your preferred platform for running Jupyter Notebooks<br>
*(For instance, you can opt for the [WekEO platform](https://help.wekeo.eu/en/articles/6337538-what-is-the-wekeo-jupyterhub))*<br>
2. Next, copy this GitHub repository or [the original one](https://github.com/eea/clms-hrsi-arlie-use-case.git) and paste all the included notebooks onto your chosen platform<br>
3. Make sure that you have the [clms_hrsi_arlie_downloader.py](clms_hrsi_arlie_downloader.py) located next to this notebook<br>
*You can see an example of the folder structure below*<br>
<img src="images/folders1.png"/><br>


4. Make sure you have installed all the necessary Python libraries. Additional to some standard Python libraries you will need: 
    - [Folium](https://pypi.org/project/folium/): A library that simplifies the creation of interactive maps using Python and Leaflet.js, allowing for easy visualization of geospatial data., 
    - [Geopandas](https://pypi.org/project/geopandas/): An extension of pandas designed to work with geospatial data, enabling operations like spatial joins and geometric manipulations using familiar DataFrame structures,
    - [Shapely](https://pypi.org/project/shapely/): A library for manipulating and analyzing planar geometric objects, providing tools for creating and working with points, lines, and polygons in a geometric context..<br>
<br>

_Other python libraries that could be quite be handy_:
- *[pandas](https://pandas.pydata.org/)*: A powerful library for data manipulation and analysis, ideal for handling structured data through DataFrames.
  
- *[matplotlib](https://matplotlib.org/)*: A versatile library for creating a wide range of static, animated, and interactive visualizations in Python, focusing on 2D plotting.

- *[pyplot](https://matplotlib.org/stable/api/pyplot_summary.html)*: A module within `matplotlib` that provides a MATLAB-like interface for creating plots easily and interactively.

- *[contextily](https://contextily.readthedocs.io/en/stable/)*: A library used to add basemaps (like satellite imagery or street maps) to geospatial visualizations, especially with `GeoPandas`.


In case you miss any libraries throughout this tutorial, simply install them by running `%pip install {package name}`
See example below:

In [ ]:
# press shift+enter to execute a cell

%pip install folium
%pip install Geopandas
%pip install Shapely

# to uncomment a line, select the line and press ctrl+/ or delete the # and space
%pip install pandas
%pip install matplotlib
%pip install pyplot
%pip install contextily

#### **Step 2: Pick your AOI**
To access ARLIE data for your specific AOI, you have two options. You can either upload a predefined geometry (e.g., a shapefile or GeoJSON) or you can draw a custom geometry on a map following the steps below. 

- If you have a [predetermined geometry, jump directly to step 3](#step-3-transform-to-wkt-format). 

- In case you [have a file in WKT format, jump to step 4](#step-4-request-arlie-data).

The geometry is used to request all lake and river polygons from the EU Hydro database within your AOI, which, in turn, is used to retrieve ARLIE data from the PostGIS database.


#### *2.1: Draw a custom geometry using Folium*

When you execute the cell below, an interactive map will appear. You can use this map to draw your Area of Interest (AOI). Follow these steps:

1. *Navigate the Map*: Use the toolbar on the left to zoom in, zoom out, and move around the map.
2. *Draw Your AOI*: Click the drawing tool (polygon) in the toolbar and draw your desired AOI on the map.
3. *Edit Geometry*: If needed, use the toolbar to edit or refine the drawn geometry.
4. *Export the Geometry*: Once you're satisfied with your AOI, click the small 'Export' button on the right side of the toolbar. A window will appear prompting you to save the file.
5. *Save as GeoJSON*: Save the file in GeoJSON format. You can save it either in the same folder as this notebook or anywhere on your computer.
6. *Upload the GeoJSON*: If you saved the file elsewhere, make sure to upload it into the same folder where this notebook is located to ensure the notebook can access it.
<br>
<br>


<img src="images/folium1.png" alt="Drawing" style="width: 800px;"/>

🤫 Note, the geometry should have an appropriate size and complexety, so try with a smaller area if you experience issues later in the workflow.

In [2]:
# Import relevant libraries
import folium
from folium.plugins import Draw

# Initialise an interactive map, get the coordinates from Google Maps for example to center the map
m = folium.Map(location=[64.5, -18], zoom_start=7)

#  Add a function to draw and export your polygon
draw = Draw(export=True, filename='aoi.geojson', position='topleft')
draw.add_to(m)

# Display the map
m

##### *2.2: Import and display you rgeometry*

Once you have saved the GeoJSON file of your AOI in the *data folder*/*, follow the steps below to import and display it. If you already have a GeoJSON file from another source, make sure to place it in the *data folder* and follow the same steps.

Steps:
1. Import the AOI Geometry: The code will read the GeoJSON file from the data folder and convert it into a geospatial format that can be displayed.

    - If you created the GeoJSON in the previous step, it should already be saved in the data folder.
    - If you already have a GeoJSON file from somewhere else, ensure it is placed in the data folder as aoi.geojson or adjust the file path accordingly.

2. Display the AOI: The map will show the outline of your selected area on an interactive map.


In [25]:
# Import relevant libraries
import folium 
import geopandas as gpd
from pathlib import Path # alternative to os.path

# Create the 'data' folder if it doesn't exist using Path
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

# Set the path to your AOI using Path
aoi_path = data_dir / 'aoi.geojson'

# Read the AOI as a GeoDataFrame
gdf = gpd.read_file(aoi_path)


# Retrieve the center points of the AOI for map centering 
aoiX = gdf["geometry"].centroid.x
aoiY = gdf["geometry"].centroid.y


# Display the AOI on a map
m = folium.Map(
    location=[aoiY[0], aoiX[0]],
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=1,
    max_zoom=15,
    control_scale=True
)

# Add the AOI to the map
folium.GeoJson(str(aoi_path), name="AOI", style_function=lambda x: {
    'fillOpacity': 0.1,
    'color': 'Crimson'
}).add_to(m)

# Show the map
m

C:\Users\michaela.hrabalikova\AppData\Local\Temp\ipykernel_47028\1576926829.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiX = gdf["geometry"].centroid.x
C:\Users\michaela.hrabalikova\AppData\Local\Temp\ipykernel_47028\1576926829.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiY = gdf["geometry"].centroid.y


#### Step 3: Transform to WKT format
Transform your AOI from the Geojson to WKT format using Shapely. In case you uploaded a shapefile, simply use this as input.

In [26]:
# Import relevant libraries
import json
from shapely.geometry import shape

# Set the path to your AOI
aoi = "data/aoi.geojson"

# Use Shapely to transform to WKT format and save it
with open(Path(aoi)) as fin, open(Path(aoi).with_suffix(".wkt"), "w") as fout:
    features = json.load(fin)["features"]
    for feature in features:
        geo = shape(feature["geometry"])
        wkt = geo.wkt
        fout.write(wkt + "\n")

In [28]:
# A WKT (Well-Known Text) is a scheme for writing a geometry into a standard text string, as you can see below
wkt

'POLYGON ((-19.659868 63.984371, -19.659868 64.450273, -18.15734 64.450273, -18.15734 63.984371, -19.659868 63.984371))'

#### Step 4: Request ARLIE data
The retrieval of the ARLIE data can take a while, depending on the size and complexety of your AOI. In the end, you should see a message informing on where the output has been saved and how many ARLIE records have been found.

In [38]:
# Import the API to download ARLIE data
from clms_hrsi_arlie_downloader import download_arlie_products

# Set the following variables to retrieve ARLIE data:
geometryWkt = wkt               # The WKT you have generated above, or simply copy+paste a geometry directly (eg. 'POLYGON ((9.667968999999999 52.48278, 10.722656 51.289406, 6.855469 49.496675, 10.283203 47.457809, 15.205078 48.922499, 14.238281 51.344339, 9.667968999999999 52.48278))')
outputDir = "./output"          # Your preferred output folder
startDate = '2023-01-01'        # Start date for ARLIE data
completionDate = '2024-09-01'   # End date for ARLIE data
cloudCoverageMax = 100          # Maximum cloud cover to consider
requestGeometries = True        # True returns polygons from EU Hydro, False only returns the ARLIE data
returnMode = 'csv_and_variable' # choose between "csv", "csv_and_variable", "variable"

geom, arlie = download_arlie_products(
    returnMode, 
    outputDir=outputDir, 
    startDate=startDate, 
    completionDate=completionDate, 
    geometryWkt=geometryWkt, 
    cloudCoverageMax=cloudCoverageMax, 
    requestGeometries=requestGeometries)

Executing request for geometries: https://cryo.land.copernicus.eu/arlie/get_geometries?geometrywkt=POLYGON+((-19.659868+63.984371%2C+-19.659868+64.450273%2C+-18.15734+64.450273%2C+-18.15734+63.984371%2C+-19.659868+63.984371))&getonlyids=True
Executing request for geometries: https://cryo.land.copernicus.eu/arlie/get_geometries?geometrywkt=POLYGON+((-19.659868+63.984371%2C+-19.659868+64.450273%2C+-18.15734+64.450273%2C+-18.15734+63.984371%2C+-19.659868+63.984371))
Writing geometries in d:\Git\Copernicus\Land\clms-hrsi-arlie_demo-iceland\output\geometries.csv
Executing request for ARLIE: https://cryo.land.copernicus.eu/arlie/get_arlie?geometrywkt=POLYGON+((-19.659868+63.984371%2C+-19.659868+64.450273%2C+-18.15734+64.450273%2C+-18.15734+63.984371%2C+-19.659868+63.984371))&cloudcoveragemax=100&startdate=2023-01-01&completiondate=2024-09-01
Done
Writing ARLIE in d:\Git\Copernicus\Land\clms-hrsi-arlie_demo-iceland\output\arlie.csv
Found 54053 ARLIE products.
Writing metadata link into d:\Git

#### Step 6: Display ARLIE data
We now retrieved the ARLIE data in a csv format and saved it to the folder <code style="background:yellow;color:black">output</code>.
The data should look similar to this when opening as a list:
> [[1, 46493, '2019-01-01T17:16:05', 27, 56, 0, 0, 17 ,1 ,'Sentinel-1 Sentinel-2'],
 [2, 46493, '2019-01-02T17:07:26', 62, 23, 0, 0, 15, 1, 'Sentinel-1 Sentinel-2']], [3, 46493, '2019-01-03T16:59:52', 51, 35, 0, 0, ...

Pass the statistics into a Pandas DataFrame to easier explore and organise the data by running the cell below.

In [39]:
# Import relevant libraries
import pandas as pd
from datetime import datetime
import os

# Parse the orignial ARLIE date format to something more readable
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')

# Read the csv file into a dataframe and display it
arlie = pd.read_csv(os.path.join(outputDir, 'arlie.csv'), parse_dates=['datetime'], delimiter=";")
arlie.to_csv(os.path.join(outputDir, 'arlie.csv'))
arlie

,id,river_km_id,datetime,water_perc,ice_perc,other_perc,cloud_perc,nd_perc,qc,source
0,1,112902,2023-01-04 07:42:36,82,4,0,0,14,2,Sentinel-1
1,2,112902,2023-01-04 18:59:49,85,0,0,0,15,2,Sentinel-1
2,3,112902,2023-01-06 18:43:30,62,24,0,0,14,2,Sentinel-1
3,4,112902,2023-01-11 18:51:41,74,4,0,0,22,2,Sentinel-1
4,5,112902,2023-01-16 07:42:35,82,3,0,0,15,2,Sentinel-1
...,...,...,...,...,...,...,...,...,...,...
54048,54049,119165,2024-08-14 18:59:51,49,39,0,0,12,0,Sentinel-1 Sentinel-2
54049,54050,119165,2024-08-16 18:43:32,37,45,1,0,17,0,Sentinel-1 Sentinel-2
54050,54051,119165,2024-08-21 18:51:44,49,41,0,0,10,0,Sentinel-1
54051,54052,119165,2024-08-26 18:59:51,48,30,3,0,19,0,Sentinel-1 Sentinel-2


The output should now look similar to the table above, with the following columns:<p>

>**id** = unique identifcation number for each ARLIE record <br>
**river_km_id** = identification number for the lake or river section polygon from the EU Hydro database<br>
**datetime** = date and time of ARLIE record<br>
**water_perc** = amount of water per waterbody in %<br>
**ice_perc** = amount of ice or snow cover per waterbody in %<br>
**other_perc** = amount of other features per waterbody in %<br>
**cloud_perc** = cloud cover per waterbody in %<br>
**nd_perc** = no data pixels per waterbody %<br>
**source** = satellite data used to derive ARLIE record<br>
**qc** = quality control value with the following quality levels:
>> high quality: 0.5 > ARLIE_confidence ≥ 0<br>
>> medium quality: 1.5 > ARLIE_confidence ≥ 0.5<br>
>> low quality: 2.5 > ARLIE_confidence ≥ 1.5<br>
>> minimal quality: 3.0 ≥ ARLIE_confidence ≥ 2.5<br>
    
To organise the data, you can sort it by date and filter it for the highest quality level. Do do this, run the cell below.

In [40]:
# Sort records by datetime
arlie = arlie.sort_values(by="datetime", ascending=True)

# Only get records with high confidence (i.e. QC flag = 0)
arlie = arlie.where(arlie.qc == 0).dropna()
display(arlie)

,id,river_km_id,datetime,water_perc,ice_perc,other_perc,cloud_perc,nd_perc,qc,source
50812,50813.0,119082.0,2023-01-04 07:42:36,78.0,15.0,0.0,0.0,7.0,0.0,Sentinel-1
51560,51561.0,119109.0,2023-01-04 07:42:36,9.0,78.0,0.0,0.0,13.0,0.0,Sentinel-1
52614,52615.0,119153.0,2023-01-04 07:42:36,46.0,48.0,0.0,0.0,6.0,0.0,Sentinel-1
52844,52845.0,119156.0,2023-01-04 07:42:36,70.0,25.0,0.0,0.0,5.0,0.0,Sentinel-1
53100,53101.0,119157.0,2023-01-04 07:42:36,3.0,86.0,0.0,0.0,11.0,0.0,Sentinel-1
...,...,...,...,...,...,...,...,...,...,...
51792,51793.0,119109.0,2024-08-28 18:43:32,70.0,15.0,0.0,0.0,15.0,0.0,Sentinel-1 Sentinel-2
51324,51325.0,119083.0,2024-08-28 18:43:32,93.0,2.0,0.0,0.0,5.0,0.0,Sentinel-1 Sentinel-2
51067,51068.0,119082.0,2024-08-28 18:43:32,80.0,13.0,0.0,0.0,7.0,0.0,Sentinel-1 Sentinel-2
53822,53823.0,119163.0,2024-08-28 18:43:32,3.0,79.0,0.0,0.0,18.0,0.0,Sentinel-1 Sentinel-2


### Step 7: Display EU Hydro data

Along with the ARLIE statistics, we retrieved EU Hydro data in csv format. Those contain vector polygon data with information about the lake/river ID, lake/river basin name, EU Hydro ID, object name, area, and river kilometre from the mouth (only for rivers which are split into 10 km long sections). Note, that we only recieve the individual EU Hydro ID numbers and no river/lake names. Read more about the full EU Hydro dataset and retrieve the river/lake names [here](#https://www.eea.europa.eu/en/datahub/datahubitem-view/2e782ca5-c7b2-4b48-8928-03031b642176).

By passing the csv file into a Pandas GeoDataFrame we can display the geometries on a folium map. Run the cells below to see how this works.

In [47]:
from shapely.wkt import loads

# Read the geometries from csv file
geometries = pd.read_csv(os.path.join(outputDir, 'geometries.csv'), delimiter=";", encoding='ISO-8859-1')

# Convert the WKT geometry strings to Shapely geometry objects
geometries['geometry'] = geometries['geometry'].apply(loads)

# Create a GeoDataFrame. As the ARLIE data comes in the LAEA European projection, we use the crs EPSG:3035
EUhydro_gdf = gpd.GeoDataFrame(geometries, geometry='geometry', crs='EPSG:3035')
EUhydro_gdf.head()

,id,geometry,basin_name,eu_hydro_id,object_nam,area,river_km
0,113203,"MULTIPOLYGON (((2954762.083 4844013.343, 29547...",Iceland,IW23002411,UNK,99594.20,NaN
1,113207,"MULTIPOLYGON (((2951558.704 4843927.164, 29516...",Iceland,IW23002415,Ampapollur,198812.77,NaN
2,113226,"MULTIPOLYGON (((2953361.428 4841883.352, 29533...",Iceland,NaN,ÃÅnÃ½tavatn fremra,96519.92,NaN
3,113225,"MULTIPOLYGON (((2955103.511 4842457.003, 29550...",Iceland,NaN,ÃÅnÃ½tavatn,869701.61,NaN
4,113223,"MULTIPOLYGON (((2955975.991 4844099.427, 29560...",Iceland,NaN,Litla-SkÃ¡lavatn,125774.21,NaN


In [37]:
# Set the path to your AOI in geojson format and read it as GeoDataFrame
aoi = "data/aoi.geojson"
aoi_gdf = gpd.read_file(aoi)

# Retrieve the center points of the AOI to display it later
aoiX = aoi_gdf["geometry"].centroid.x
aoiY = aoi_gdf["geometry"].centroid.y

# Create an empty map
m = folium.Map(
    location=[aoiY[0],aoiX[0]],
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=1,
    max_zoom=15,
    control_scale=True
)

# Add the EU Hydro data and your AOI to an interactive map and display it
folium.GeoJson(EUhydro_gdf, name="EU Hydro rivers & Lakes", style_function=lambda x: {'color': 'RoyalBlue'}).add_to(m)
folium.GeoJson(aoi, name="AOI", style_function=lambda x: {'fillOpacity': 0.1,'color': 'Crimson'}).add_to(m)

folium.LayerControl().add_to(m)

m

C:\Users\michaela.hrabalikova\AppData\Local\Temp\ipykernel_47028\1810828181.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiX = aoi_gdf["geometry"].centroid.x
C:\Users\michaela.hrabalikova\AppData\Local\Temp\ipykernel_47028\1810828181.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoiY = aoi_gdf["geometry"].centroid.y


Awesome! 🥳 You now requested ARLIE data and the assoiciated EU Hydro polygons for your AOI! We will continue in the next notebook to show you how to manipulate data. 